# Final Project - Dashboard

Names:

- Milagros Contreras Elizalde
  - *e-mail*: milagros.contreras8430@alumnos.udg.mx

- Heriberto Zapata Solano
  - *e-mail*: heriberto.zapata7939@alumnos.udg.mx

# Modules

In [2]:
import numpy as np
import pandas as pd
import requests

We download a CSV file from Google Drive and load it into a pandas DataFrame for further analysis. The file contains data related to shipments, with 10,999 observations and 12 variables, including customer ID, warehouse block, mode of shipment, customer rating, etc.

In [3]:
url = 'https://drive.google.com/uc?export=download&id=1-YHgns9U6ehZcwsOcFC-MNMny3oYsgPp'
response = requests.get(url)
open('shipping.csv', 'wb').write(response.content)
df = pd.read_csv('shipping.csv')
df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


**Variables**
- ID: ID Number of Customers.
- Warehouse block: The Company have big Warehouse which is divided in to block such as A,B,C,D,E.
- Mode of shipment:The Company Ships the products in multiple way such as Ship, Flight and Road.
- Customer care calls: The number of calls made from enquiry for enquiry of the shipment.
- Customer rating: The company has rated from every customer. 1 is the lowest (Worst), 5 is the highest (Best).
- Cost of the product: Cost of the Product in US Dollars.
- Prior purchases: The Number of Prior Purchase.
- Product importance: The company has categorized the product in the various parameter such as low, medium, high.
- Gender: Male and Female.
- Discount offered: Discount offered on that specific product.
- Weight in gms: It is the weight in grams.
- Reached on time: It is the target variable, where 1 Indicates that the product has NOT reached on time and 0 indicates it has reached on time.

In [34]:
import numpy as np
import pandas as pd
import dash
from dash import dcc, html
import plotly.express as px
from dash.dependencies import Input, Output

# Initialize the dataset
df = pd.read_csv('shipping.csv')

# Initialize the Dash application
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    # Centered title
    html.Div(
        html.H1("Shipment Dashboard - On-Time Analysis", style={'text-align': 'center'}),
        style={'margin-bottom': '20px', 'width': '100%', 'display': 'flex', 'justify-content': 'center'}
    ),

    # Main layout: Flexbox for horizontal arrangement
    html.Div(style={'display': 'flex', 'align-items': 'flex-start'}),
    
    # Filters - Parameter Menu
    html.Div([
        html.Label("Warehouse Block :"),
        dcc.Dropdown(
            id='warehouse-block',
            options=[{'label': block, 'value': block} for block in df['Warehouse_block'].unique()],
            multi=True
        ),
        html.Label("Mode of Shipment:"),
        dcc.Dropdown(
            id='mode-shipment',
            options=[{'label': mode, 'value': mode} for mode in df['Mode_of_Shipment'].unique()],
            multi=True
        ),
        html.Label("Customer Rating:"),
        dcc.RangeSlider(
            id='customer-rating',
            min=df['Customer_rating'].min(),
            max=df['Customer_rating'].max(),
            step=1,
            value=[df['Customer_rating'].min(), df['Customer_rating'].max()],
            marks={i: str(i) for i in range(1, 6)}
        ),
        html.Label("Product Importance:"),
        dcc.Dropdown(
            id='product-importance',
            options=[{'label': importance, 'value': importance} for importance in df['Product_importance'].unique()],
            multi=True
        ),
    ], style={'padding': '10px', 'width': '30%', 'display': 'inline-block', 'vertical-align': 'top'}),
    
    # Graphs and Metrics
    html.Div([
        # Metrics
        html.Div([
            html.H3("Metrics"),
            html.Div(id='metrics-output'),  # This is where the metrics will be displayed
        ], style={'padding': '10px', 'display': 'inline-block', 'vertical-align': 'top'}),
        
        # Chart type selection
        html.Div([
            html.Label("Chart Type:"),
            dcc.RadioItems(
                id='chart-type',
                options=[
                    {'label': 'Bar', 'value': 'bar'},
                    {'label': 'Line', 'value': 'line'},
                    {'label': 'Scatter', 'value': 'scatter'}
                ],
                value='bar'
            ),
        ], style={'padding': '10px', 'display': 'inline-block', 'vertical-align': 'top'}),
        
        # Shipment graph
        dcc.Graph(id='shipment-graph', style={'height': '60vh'}),
    ], style={'width': '70%', 'display': 'inline-block', 'vertical-align': 'top', 'order': '1'})
], style={'display': 'flex', 'flex-direction': 'column', 'align-items': 'flex-start'})  # Align everything to the top

# Callback to update the graph and metrics based on the filters and chart type
@app.callback(
    [Output('shipment-graph', 'figure'),
     Output('metrics-output', 'children')],
    [Input('warehouse-block', 'value'),
     Input('mode-shipment', 'value'),
     Input('customer-rating', 'value'),
     Input('product-importance', 'value'),
     Input('chart-type', 'value')]
)
def update_dashboard(warehouse_block, mode_shipment, customer_rating, product_importance, chart_type):
    # Filter the dataset
    filtered_df = df.copy()
    if warehouse_block:
        filtered_df = filtered_df[filtered_df['Warehouse_block'].isin(warehouse_block)]
    if mode_shipment:
        filtered_df = filtered_df[filtered_df['Mode_of_Shipment'].isin(mode_shipment)]
    if customer_rating:
        filtered_df = filtered_df[(filtered_df['Customer_rating'] >= customer_rating[0]) &
                                  (filtered_df['Customer_rating'] <= customer_rating[1])]
    if product_importance:
        filtered_df = filtered_df[filtered_df['Product_importance'].isin(product_importance)]

    # Calculate metrics
    total_shipments = len(filtered_df)
    on_time_shipments = len(filtered_df[filtered_df['Reached.on.Time_Y.N'] == 0])
    late_shipments = total_shipments - on_time_shipments
    on_time_percentage = round((on_time_shipments / total_shipments) * 100, 2) if total_shipments > 0 else 0
    avg_customer_rating = filtered_df['Customer_rating'].mean() if total_shipments > 0 else 0

    # Create metrics output
    metrics = [
        html.P(f"Total Shipments: {total_shipments}"),
        html.P(f"On-Time Shipments: {on_time_shipments}"),
        html.P(f"Late Shipments: {late_shipments}"),
        html.P(f"On-Time Percentage: {on_time_percentage}%"),
        html.P(f"Average Customer Rating: {round(avg_customer_rating, 2)}")
    ]

    # Create shipment graph
    if chart_type == 'bar':
        fig = px.bar(filtered_df, x='Mode_of_Shipment', color='Reached.on.Time_Y.N', 
                     barmode='group', title="On-Time Shipments by Mode of Shipment")
    elif chart_type == 'line':
        fig = px.line(filtered_df, x='Mode_of_Shipment', y='Reached.on.Time_Y.N', 
                      title="On-Time Shipments by Mode of Shipment")
    else:
        fig = px.scatter(filtered_df, x='Mode_of_Shipment', y='Customer_rating', 
                         color='Reached.on.Time_Y.N', title="On-Time Shipments and Customer Rating")

    return fig, metrics

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)

The code defines the layout of the web application using Dash components. The layout consists of several components, including:
- A centered title
- A filter menu for selecting features from the dataset
- An interactive chart for visualizing the data
- A metrics panel for displaying statistics about the data

The filter menu allows the user to select characteristics of the dataset. This menu includes options to select the warehouse block, shipping mode, customer rating, and product importance. 

This chart updates dynamically based on user selections in the filter menu.

*Metrics Dashboard*

The metrics dashboard displays statistics about the data. This panel is updated dynamically based on the user's selections in the filter menu.

*Callback*

The callback is a function that is executed when the user selects options in the filter menu. This function updates the interactive chart and the metrics dashboard based on user selections.

*Calculating metrics*

The update_dashboard function calculates metrics for the metrics dashboard.

Database infromation: https://www.kaggle.com/datasets/nayanack/shipping

**Note:**  The company has rated from every customer. 1 is the lowest (Worst), 5 is the highest (Best).